In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import csv
import kagglehub
import torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from tqdm.notebook import tqdm
import re

# **1. Предобработка данных**


In [ ]:
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
path = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")

100%|██████████| 1.06G/1.06G [00:10<00:00, 114MB/s] 


Extracting files...


In [ ]:
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1


In [ ]:
reviews_file_path = '/root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1/Books_rating.csv'
books_details_file_path = '/root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1/books_data.csv'

# Load the reviews file
reviews_df = pd.read_csv(reviews_file_path)

# Load the Books Details file
books_details_df = pd.read_csv(books_details_file_path)

In [ ]:
print("Reviews Dataset:")
print(reviews_df.info())

print("\nBooks Details Dataset:")
print(books_details_df.info())

Reviews Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB
None

Books Details Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 


**Предобработка данных**

In [ ]:
# Проверка на пропущенные значения
reviews_missing = reviews_df.isnull().sum()

books_missing = books_details_df.isnull().sum()

print("Reviews Missing Values:")
print(reviews_missing)

print("\nBooks Details Missing Values:")
print(books_missing)

Reviews Missing Values:
Id                          0
Title                     208
Price                 2518829
User_id                561787
profileName            561905
review/helpfulness          0
review/score                0
review/time                 0
review/summary            407
review/text                 8
dtype: int64

Books Details Missing Values:
Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64


In [ ]:
# Удаляем строки с отсутствующими значениями в столбцах 'Title' и 'User_id'
reviews_df = reviews_df.dropna(subset=['Title', 'User_id'])

# Удаляем столбец 'profileName', так как он не будет использоваться
reviews_df = reviews_df.drop(columns=['profileName'])

# Удаляем столбец 'Price', так как он не будет использоваться
reviews_df = reviews_df.drop(columns=['Price'])

# Заполняем пропущенные значения в столбцах 'review/summary' и 'review/text' пустыми строками
reviews_df['review/summary'] = reviews_df['review/summary'].fillna('')
reviews_df['review/text'] = reviews_df['review/text'].fillna('')

#Replacing NaN values with 'Unknown' string
books_details_df['description'].fillna('Unknown', inplace = True)
books_details_df['authors'].fillna('Unknown', inplace = True)
books_details_df['image'].fillna('Unknown', inplace = True)
books_details_df['previewLink'].fillna('Unknown', inplace = True)
books_details_df['publisher'].fillna('Unknown', inplace = True)
books_details_df['publishedDate'].fillna('Unknown', inplace = True)
books_details_df['infoLink'].fillna('Unknown', inplace = True)
books_details_df['categories'].fillna('Unknown', inplace = True)

# Отображаем обновленную информацию о пропущенных значениях
reviews_missing_values = reviews_df.isnull().sum()
print("Пропущенные значения в данных об отзывах после обработки:")
print(reviews_missing_values)


<ipython-input-8-e6686a383af4>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books_details_df['description'].fillna('Unknown', inplace = True)
<ipython-input-8-e6686a383af4>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

Пропущенные значения в данных об отзывах после обработки:
Id                    0
Title                 0
User_id               0
review/helpfulness    0
review/score          0
review/time           0
review/summary        0
review/text           0
dtype: int64


In [ ]:
# Удаляем строки с отсутствующими значениями в столбце 'Title'
books_details_df = books_details_df.dropna(subset=['Title'])

# Заполняем пропущенные значения в столбце 'ratingsCount' медианным значением
books_details_df['ratingsCount'] = books_details_df['ratingsCount'].fillna(books_details_df['ratingsCount'].median())

# Заполняем пропущенные значения в текстовых столбцах пустыми строками
textual_columns = ['description', 'authors', 'publisher', 'publishedDate', 'categories']
books_details_df[textual_columns] = books_details_df[textual_columns].fillna('')

# Удаляем столбцы, которые не планируем использовать
columns_to_drop = ['image', 'previewLink', 'infoLink']
books_details_df = books_details_df.drop(columns=columns_to_drop)

# Отображаем обновленную информацию о пропущенных значениях
books_details_missing_values = books_details_df.isnull().sum()
print("Пропущенные значения в данных о книгах после обработки:")
print(books_details_missing_values)


<ipython-input-9-18ed9738d991>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_details_df['ratingsCount'] = books_details_df['ratingsCount'].fillna(books_details_df['ratingsCount'].median())


Пропущенные значения в данных о книгах после обработки:
Title            0
description      0
authors          0
publisher        0
publishedDate    0
categories       0
ratingsCount     0
dtype: int64


**Объединение** **датасетов**

In [ ]:
#Merging the dataframes
merged = pd.merge(books_details_df, reviews_df, on = 'Title')

In [ ]:
merged.head(5)

,Title,description,authors,publisher,publishedDate,categories,ratingsCount,Id,User_id,review/helpfulness,review/score,review/time,review/summary,review/text
0,Its Only Art If Its Well Hung!,Unknown,['Julie Strain'],Unknown,1996,['Comics & Graphic Novels'],2.0,1882931173,AVCGYZL8FQQTD,7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,2005-01-01,['Biography & Autobiography'],2.0,0826414346,A30TK6U7DNS82R,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,2005-01-01,['Biography & Autobiography'],2.0,0826414346,A3UH4UZ4RSVO82,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,2005-01-01,['Biography & Autobiography'],2.0,0826414346,A2MVUWT453QH61,7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,2005-01-01,['Biography & Autobiography'],2.0,0826414346,A22X4XUPKF66MR,3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [ ]:
# Анализируем объединенный DataFrame на наличие пропущенных значений
merged_df_missing_values = merged.isnull().sum()
print("Пропущенные значения в объединенном DataFrame:")
print(merged_df_missing_values)

Пропущенные значения в объединенном DataFrame:
Title                 0
description           0
authors               0
publisher             0
publishedDate         0
categories            0
ratingsCount          0
Id                    0
User_id               0
review/helpfulness    0
review/score          0
review/time           0
review/summary        0
review/text           0
dtype: int64


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model_path = 'sentiment_model.pth'

class SentimentModel(nn.Module):
    def __init__(self, input_size, hidden_size=32,dropout_rate=0.4):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate), #Dropout is a regularization technique commonly used in neural networks during training to prevent overfitting
            nn.Linear(hidden_size, hidden_size*2), #  linear layer (y=mx+c)
            nn.ReLU(), # max(0,x) , basically adds non-linearity
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size*2, hidden_size*4),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size*4, hidden_size*8),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size*8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.sequential(x)


# Load the entire model
sentiment_model=torch.load(model_path)
sentiment_model.to(device)

<ipython-input-15-cade8fc58460>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sentiment_model=torch.load(model_path)


SentimentModel(
  (sequential): Sequential(
    (0): Linear(in_features=12000, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=32, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=64, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.4, inplace=False)
    (9): Linear(in_features=128, out_features=256, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=256, out_features=1, bias=True)
    (13): Sigmoid()
  )
)

In [ ]:
from torchtext.data.utils import get_tokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import joblib
import re

import nltk
nltk.download('stopwords', download_dir='/root/nltk_data')

class SentimentAnalysisDataset(torch.utils.data.Dataset):
    def __init__(self, texts, vectorizer, sentiment_model, device):
        """
        A dataset for sentiment analysis.

        Args:
            texts (list): List of text reviews.
            vectorizer: The vectorizer used during training.
            sentiment_model: The trained sentiment analysis model.
            device: The device (CPU/GPU) to which the tensor should be sent.
        """
        self.texts = texts
        self.vectorizer = vectorizer
        self.sentiment_model = sentiment_model
        self.device = device

    def __len__(self):
        """
        Returns the total number of samples in the dataset.

        Returns:
            int: The number of samples in the dataset.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Retrieves and processes a single sample from the dataset.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            torch.Tensor: The sentiment score for the input text.
        """
        text = self.texts[idx]
        text = text.lower() # Convert to lowercase
        text = re.sub(r'<br\s*/*\s*><br\s*/*\s*>', '', text)
        text = re.sub(r"[^a-zA-Z0-9]", " ", text) # Remove special characters
        tokenizer = get_tokenizer("basic_english")
        tokens = tokenizer(text) # Tokenization
        tokens = [word for word in tokens if word.isalpha()] # Remove non-alphabetic tokens
        stop_words = set(stopwords.words("english")) - {'not'}
        tokens = [word for word in tokens if word not in stop_words] # Remove stopwords

        # Use the same vectorizer used during training
        vectorized_text = self.vectorizer.transform([" ".join(tokens)])
        tensor = torch.tensor(vectorized_text.toarray(), dtype=torch.float32)
        tensor = tensor.to(self.device)

        with torch.inference_mode():
            score = self.sentiment_model(tensor)
        return score

# Load the vectorizer
vectorizer = joblib.load('vectorizer.pkl')

# Batch size for processing
batch_size = 10000

dataset = SentimentAnalysisDataset(merged["review/text"], vectorizer, sentiment_model, device)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

if 'sentiment_score' not in merged.columns:
    merged['sentiment_score'] = None

# __getitem__ is automatically(implicitely) called when using for loops
for i, scores in tqdm(enumerate(loader), total=len(loader), desc="Processing Reviews"):
    start = i * batch_size
    for j, score in enumerate(scores):
        score = score.item()
        merged.at[start+j, 'sentiment_score'] = score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
merged['sentiment_label'] = (merged['sentiment_score'] >= 0.5).astype(int)
merged.to_csv('merged_dataframe_with_sentiment_labels.csv', index=False)

In [ ]:
merged_dataframe_pth="/content/drive/MyDrive/merged_dataframe_with_sentiment_labels.csv"
merged= pd.read_csv(merged_dataframe_pth)
merged=merged.fillna('')

In [ ]:
merged[['sentiment_score','review/text','sentiment_label']][:5]
#print(merged_df['review/text'].loc[0])

,sentiment_score,review/text,sentiment_label
0,0.274792,This is only for Julie Strain fans. It's a col...,0
1,0.355799,I don't care much for Dr. Seuss but after read...,0
2,0.989629,"If people become the books they read and if ""t...",1
3,0.095767,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",0
4,0.916031,Philip Nel - Dr. Seuss: American IconThis is b...,1


In [ ]:
merged.columns

Index(['Title', 'description', 'authors', 'publisher', 'publishedDate',
       'categories', 'ratingsCount', 'Id', 'User_id', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text',
       'sentiment_score', 'sentiment_label'],
      dtype='object')

# **2. Построение модели**

### **Рекомендательная система**

***1. Коллаборативная фильтрация***

Коллаборативная фильтрация предлагает элементы на основе предпочтений пользователей или схожих пользователей.

Существует два основных типа коллаборативной фильтрации:

1.1. **Основанная на пользователях (User-Based Collaborative Filtering):**

* **Идея:** Рекомендует элементы, которые понравились пользователям со схожими предпочтениями.
* **Процесс:** Выявляются пользователи с похожими предпочтениями, и пользователю предлагаются элементы, которые понравились этим похожим пользователям.
* **Матрица:** Используется матрица взаимодействия "пользователь-элемент" для поиска схожести между пользователями.

1.2. **Основанная на элементах (Item-Based Collaborative Filtering):**

* **Идея:** Рекомендует элементы, похожие на те, с которыми взаимодействовал пользователь.
* **Процесс:** Выявляются элементы, схожие с теми, которые уже были отмечены пользователем, и рекомендуются аналогичные элементы.
* **Матрица:** Используется транспонированная матрица взаимодействия "пользователь-элемент" для поиска схожести между элементами.

Пример: В фильтрации, основанной на пользователях, выходом будет список пользователей, а в фильтрации, основанной на элементах, — список предметов (например, книг).

***2. Контентная фильтрация (Content-Based Filtering)***

Контентная фильтрация рекомендует элементы, основываясь на их характеристиках и предпочтениях пользователя. Для книг такими характеристиками могут быть жанры, авторы, оценки настроений и т.д.

**Особенности контентной фильтрации:**

* Использование таких признаков, как описание, авторы, категории, оценки настроений.
* Создание профилей элементов на основе этих признаков.

***3. Гибридный подход***

Сочетание коллаборативной и контентной фильтрации:

* Генерация рекомендаций с использованием коллаборативной фильтрации.
* Уточнение рекомендаций с помощью контентной фильтрации на основе признаков, таких как описания книг, авторы, оценки настроений.

Пример объединения:

Если модель CF рекомендует следующие книги:

* Книга A (CF-оценка: 0.8)
* Книга B (CF-оценка: 0.6)
* Книга C (CF-оценка: 0.7)

А модель CBF рассчитывает следующие оценки настроений:

* Книга A (CBF-оценка: 0.9)
* Книга B (CBF-оценка: 0.7)
* Книга C (CBF-оценка: 0.8)

Объединенные оценки будут такими:

* Книга A: 1.7
* Книга B: 1.3
* Книга C: 1.5

**Результат:** Сочетание подходов дает более персонализированные и разнообразные рекомендации.

### **Коллаборативная фильтрация**

На данном этапе для построения системы рекомендаций реализуем модель коллаборативной фильтрации, основанную на взаимодействии пользователей с объектами (например, книгами).

**Подготовка данных**

In [ ]:
# Create a mapping of user and item indices
user_ids = merged['User_id'].unique()
item_ids = merged['Title'].unique()
user_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
item_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
index_to_title = {idx: title for title, idx in item_to_index.items()}

# Convert user and item indices to tensors
user_indices = torch.tensor([user_to_index[user_id] for user_id in merged['User_id']], dtype=torch.long)
item_indices = torch.tensor([item_to_index[item_id] for item_id in merged['Title']], dtype=torch.long)
ratings = torch.tensor(merged['review/score'].values, dtype=torch.float32)

In [ ]:
# Create a DataLoader
dataset = TensorDataset(user_indices, item_indices, ratings)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True) # i want to set batch size to lower number so that during training , model can adjust its parameters more as i will try to set total epochs lower
#A lower batch size can allow the model to update its parameters more frequently within each epoch.

**Создание модели**

In [ ]:
class CollaborativeFilteringModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, hidden_dim):
        super().__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.hidden_layer = nn.Linear(embedding_dim * 2, hidden_dim)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, 1)

    def forward(self, user_indices, item_indices):
        user_embedded = self.user_embedding(user_indices)
        item_embedded = self.item_embedding(item_indices)
        concatenated = torch.cat([user_embedded, item_embedded], dim=1)
        hidden_output = self.relu(self.hidden_layer(concatenated))
        output = self.output_layer(hidden_output)
        return output

    def get_similar_titles(self, input_title_index, top_k=100):
        device = self.item_embedding.weight.device  # Get the device of the embeddings

        # Move the input title index to the same device as the model
        input_title_index = torch.tensor([input_title_index], device=device)

        # Get the embedding for the input title
        input_title_embedding = self.item_embedding(input_title_index)

        # Get embeddings for all titles
        all_title_embeddings = self.item_embedding.weight

        # Calculate cosine similarity
        similarities = F.cosine_similarity(input_title_embedding, all_title_embeddings)

        # Get indices of top-k similar titles
        #argsort returns the indices that sort a tensor along a given dimension in ascending order(default) by value.
        similar_title_indices = torch.argsort(similarities, descending=True)[:top_k]

        # Convert indices to a list of titles
        similar_titles = [index_to_title[idx.item()] for idx in similar_title_indices]
        #we are using item() to get scalar value instead of tensor which can be used as an key index for dictionary
        return similar_titles

In [ ]:
import torch.nn as nn
import torch

# Suppose we have 3 unique users and 4 unique items
num_users = 3
num_items = 4

# Embedding dimensions
embedding_dim = 2

# Create user and item embeddings
user_embeddings = nn.Embedding(num_users, embedding_dim)
item_embeddings = nn.Embedding(num_items, embedding_dim)
print(user_embeddings)
print(item_embeddings)

# Generate example user and item IDs
user_ids_ex = torch.LongTensor([0, 1, 2,2,1])  # User IDs
item_ids_ex = torch.LongTensor([2, 1, 3, 0,0])  # Item IDs

# Get embeddings for user and item IDs
user_embedded = user_embeddings(user_ids_ex)
item_embedded = item_embeddings(item_ids_ex)

print("User Embeddings:")
print(user_embedded)

print("\nItem Embeddings:")
print(item_embedded)
print("\nconcatnated:")
print(torch.cat([user_embedded, item_embedded], dim=1))

Embedding(3, 2)
Embedding(4, 2)
User Embeddings:
tensor([[-0.9695,  0.5545],
        [-0.5864,  1.2729],
        [ 0.9143, -0.2067],
        [ 0.9143, -0.2067],
        [-0.5864,  1.2729]], grad_fn=<EmbeddingBackward0>)

Item Embeddings:
tensor([[ 0.1492,  0.3529],
        [ 0.0624, -0.5065],
        [-0.4483,  0.4237],
        [-0.4816,  0.3706],
        [-0.4816,  0.3706]], grad_fn=<EmbeddingBackward0>)

concatnated:
tensor([[-0.9695,  0.5545,  0.1492,  0.3529],
        [-0.5864,  1.2729,  0.0624, -0.5065],
        [ 0.9143, -0.2067, -0.4483,  0.4237],
        [ 0.9143, -0.2067, -0.4816,  0.3706],
        [-0.5864,  1.2729, -0.4816,  0.3706]], grad_fn=<CatBackward0>)


In [ ]:
input1 = torch.randn(1, 2)
input2 = torch.randn(5, 2)
print(input1,input2)
c_s=F.cosine_similarity(input1,input2)
print(c_s)
print(torch.argsort(c_s, descending=True))

tensor([[ 0.0409, -0.4797]]) tensor([[ 2.1487, -0.4313],
        [ 0.1908, -0.2544],
        [-0.0033, -0.4456],
        [ 0.8476, -0.1232],
        [ 2.5923,  1.5001]])
tensor([ 0.2794,  0.8481,  0.9957,  0.2274, -0.4255])
tensor([2, 1, 0, 3, 4])


In [ ]:
# Instantiate the model and move it to device
num_users = len(user_ids)
num_items = len(item_ids)
embedding_dim = 100
model = CollaborativeFilteringModel(num_users, num_items, embedding_dim,32)

**Обучение модели**

In [ ]:
model.to(device)
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 3
for epoch in tqdm(range(epochs),desc="Training Collaborative Model"):
    model.train()
    total_loss = 0.0
    for user_batch, item_batch, rating_batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        user_batch, item_batch, rating_batch = user_batch.to(device), item_batch.to(device), rating_batch.to(device)
        #user_batch = user_batch.long()
        #item_batch = item_batch.long()
        optimizer.zero_grad()
        predictions = model(user_batch, item_batch).squeeze()
        loss = criterion(predictions, rating_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Loss: {average_loss:.4f}')

Training Collaborative Model:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/3:   0%|          | 0/38095 [00:00<?, ?it/s]

Epoch 1/3, Average Loss: 1.3586


Epoch 2/3:   0%|          | 0/38095 [00:00<?, ?it/s]

Epoch 2/3, Average Loss: 1.0434


Epoch 3/3:   0%|          | 0/38095 [00:00<?, ?it/s]

Epoch 3/3, Average Loss: 0.8188


In [ ]:
user_index = torch.tensor([5], dtype=torch.long).to(device)
item_index = torch.tensor([1], dtype=torch.long).to(device)
prediction = model(user_index, item_index).item()
print(f'Predicted Rating: {prediction:.4f}')

**Оценка модели**

In [ ]:
model.eval()
total_mse = 0.0

with torch.inference_mode():
    for user_batch, item_batch, rating_batch in tqdm(dataloader,desc="evaluating..."):
        user_batch, item_batch, rating_batch = user_batch.to(device), item_batch.to(device), rating_batch.to(device)
        predictions = model(user_batch, item_batch).squeeze()
        mse = nn.functional.mse_loss(predictions, rating_batch)
        total_mse += mse.item()

average_mse = total_mse / len(dataloader)
rmse = average_mse ** 0.5

print(f'Mean Squared Error: {average_mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')

evaluating...:   0%|          | 0/38095 [00:00<?, ?it/s]

Mean Squared Error: 20.5608
Root Mean Squared Error: 4.5344


**Сохранение модели**

In [ ]:
model_path = '/content/drive/MyDrive/collaborative_filtering_model.pth'

torch.save(model, model_path)
print(f'Model saved to {model_path}')

Model saved to /content/drive/MyDrive/collaborative_filtering_model.pth


In [ ]:
model_path="/content/drive/MyDrive/collaborative_filtering_model.pth"

model = torch.load(model_path, map_location=device)
model.to(device)

<ipython-input-30-f1b3d58fe375>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


CollaborativeFilteringModel(
  (user_embedding): Embedding(1008961, 100)
  (item_embedding): Embedding(206711, 100)
  (hidden_layer): Linear(in_features=200, out_features=32, bias=True)
  (relu): ReLU()
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
)

In [ ]:
model.parameters

<bound method Module.parameters of CollaborativeFilteringModel(
  (user_embedding): Embedding(1008961, 100)
  (item_embedding): Embedding(206711, 100)
  (hidden_layer): Linear(in_features=200, out_features=32, bias=True)
  (relu): ReLU()
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
)>

In [ ]:
user_index = torch.tensor([user_to_index['A30TK6U7DNS82R']], dtype=torch.long)
item_index = torch.tensor([item_to_index['The Insiders (Insiders (Bloomsbury))']], dtype=torch.long)
user_index , item_index= user_index.to(device), item_index.to(device)
prediction = model(user_index, item_index).item()
print(f'Predicted Rating: {prediction:.4f}')

Predicted Rating: 4.0501


### **Контентная фильтрация**

Для каждого элемента (книга) извлекаются такие признаки, как жанры, авторы, оценки тональности и т. д.

Эти признаки используются для построения модели, которая может рекомендовать элементы, похожие на те, что понравились пользователю.

In [ ]:
# Вывод информации о наборе данных
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438018 entries, 0 to 2438017
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Title               object 
 1   description         object 
 2   authors             object 
 3   publisher           object 
 4   publishedDate       object 
 5   categories          object 
 6   ratingsCount        float64
 7   Id                  object 
 8   User_id             object 
 9   review/helpfulness  object 
 10  review/score        float64
 11  review/time         int64  
 12  review/summary      object 
 13  review/text         object 
 14  sentiment_score     float64
 15  sentiment_label     int64  
dtypes: float64(3), int64(2), object(11)
memory usage: 297.6+ MB


In [ ]:
title_sentiment_aggregated = merged.groupby(['Title','authors','categories'])['sentiment_score'].mean().reset_index()

In [ ]:
title_sentiment_aggregated[:5]

,Title,authors,categories,sentiment_score
0,""" Film technique, "" and, "" Film acting """,['V. I. Pudovkin'],['Drama'],0.771609
1,""" We'll Always Have Paris"": The Definitive Gui...","['Robert A. Nowlan', 'Gwendolyn Wright Nowlan']",['Reference'],0.629316
2,"""... And Poetry is Born ..."" Russian Classical...",['Aleksandr Sergeevich Pushkin'],['Russian poetry'],0.907028
3,"""A Titanic hero"" Thomas Andrews, shipbuilder",['Shan F. Bullock'],Unknown,0.912956
4,"""A Truthful Impression of the Country"": Britis...","['Nicholas J. Clifford', 'Nicholas Rowland Cli...",['History'],0.231373


In [ ]:
len(title_sentiment_aggregated)

206711

In [ ]:
# Создание сопоставления уникальных категорий, авторов и названий
unique_categories = merged['categories'].unique()
unique_authors = merged['authors'].unique()
unique_titles = title_sentiment_aggregated['Title'].unique()

category_to_index = {category: idx for idx, category in enumerate(unique_categories)}
author_to_index = {author: idx for idx, author in enumerate(unique_authors)}
title_to_index = {title: idx for idx, title in enumerate(unique_titles)}

In [ ]:
# Преобразование категорий, авторов, заголовков в тензоры
category_indices = torch.tensor([category_to_index[category] for category in title_sentiment_aggregated['categories']], dtype=torch.long)
author_indices = torch.tensor([author_to_index[author] for author in title_sentiment_aggregated['authors']], dtype=torch.long)
title_indices = torch.tensor([title_to_index[title] for title in title_sentiment_aggregated['Title']], dtype=torch.long)

In [ ]:
sentiment_scores = torch.tensor(title_sentiment_aggregated['sentiment_score'].values, dtype=torch.float32)

target_variable = torch.tensor(merged.groupby('Title')['sentiment_label'].mean().values, dtype=torch.float32)

In [ ]:
cbf_dataset = TensorDataset(category_indices, author_indices, title_indices, sentiment_scores, target_variable)
cbf_dataloader = DataLoader(cbf_dataset, batch_size=64, shuffle=True)

**Создание модели**

In [ ]:
class ContentBasedFilteringModel(nn.Module):
    def __init__(self, num_categories, num_authors, num_titles, embedding_dim):
        super(ContentBasedFilteringModel, self).__init__()
        self.category_embedding = nn.Embedding(num_categories, embedding_dim)
        self.author_embedding = nn.Embedding(num_authors, embedding_dim)
        self.title_embedding = nn.Embedding(num_titles, embedding_dim)
        self.sentiment_linear = nn.Linear(4 * embedding_dim, 1)

    def forward(self, category_indices, author_indices, title_indices, sentiment_scores):
        category_embedded = self.category_embedding(category_indices)
        author_embedded = self.author_embedding(author_indices)
        title_embedded = self.title_embedding(title_indices)
        sentiment_expanded = sentiment_scores.unsqueeze(1).expand_as(category_embedded)

        concatenated = torch.cat([category_embedded, author_embedded, title_embedded, sentiment_expanded], dim=1)
        output = self.sentiment_linear(concatenated)
        return output

In [ ]:
# Создание экземпляра модели
num_categories = len(unique_categories)
num_authors = len(unique_authors)
num_titles = len(unique_titles)
embedding_dim = 64
cbf_model = ContentBasedFilteringModel(num_categories, num_authors, num_titles, embedding_dim)
cbf_model.to(device)

ContentBasedFilteringModel(
  (category_embedding): Embedding(10669, 64)
  (author_embedding): Embedding(124098, 64)
  (title_embedding): Embedding(206711, 64)
  (sentiment_linear): Linear(in_features=256, out_features=1, bias=True)
)

In [ ]:
# Определение функции потерь и оптимизатора
criterion_cbf = nn.BCEWithLogitsLoss()
optimizer_cbf = optim.Adam(cbf_model.parameters(), lr=0.001)

**Обучение модели**

In [ ]:
epochs_cbf = 8

for epoch in tqdm(range(epochs_cbf),desc='Training Content based filtering model'):
    cbf_model.train()
    total_loss_cbf = 0.0

    for category_batch, author_batch, title_batch, sentiment_batch, target_batch in tqdm(cbf_dataloader):
        category_batch, author_batch, title_batch, sentiment_batch, target_batch = category_batch.to(device), author_batch.to(device), title_batch.to(device), sentiment_batch.to(device), target_batch.to(device)
        optimizer_cbf.zero_grad()
        predictions_cbf = cbf_model(
            category_batch, author_batch, title_batch, sentiment_batch
        ).squeeze()
        loss_cbf = criterion_cbf(
            predictions_cbf, target_batch
        )
        loss_cbf.backward()
        optimizer_cbf.step()
        total_loss_cbf += loss_cbf.item()

    average_loss = total_loss_cbf / len(cbf_dataloader)
    print(f'Epoch {epoch + 1}/{epochs_cbf}, Average Loss: {average_loss:.4f}')

Training Content based filtering model:   0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3230 [00:00<?, ?it/s]

Epoch 1/8, Average Loss: 0.6580


  0%|          | 0/3230 [00:00<?, ?it/s]

Epoch 2/8, Average Loss: 0.6580


...

Epoch 6/8, Average Loss: 0.6580


  0%|          | 0/3230 [00:00<?, ?it/s]

Epoch 7/8, Average Loss: 0.6580


  0%|          | 0/3230 [00:00<?, ?it/s]

Epoch 8/8, Average Loss: 0.6580


Оценка (на данный момент не используются показатели оценки, такие как f1score)

In [ ]:
# Цикл оценки
cbf_model.eval()
total_loss_cbf = 0.0

with torch.inference_mode():
    for category_batch, author_batch, title_batch, sentiment_batch, target_batch in cbf_dataloader:
        category_batch, author_batch, title_batch, sentiment_batch, target_batch = category_batch.to(device), author_batch.to(device), title_batch.to(device), sentiment_batch.to(device), target_batch.to(device)
        predictions_cbf = cbf_model(
            category_batch, author_batch, title_batch, sentiment_batch
        ).squeeze()
        loss_cbf = criterion_cbf(
            predictions_cbf, target_batch
        )
        total_loss_cbf += loss_cbf.item()

    average_loss = total_loss_cbf / len(cbf_dataloader)
    print(f'Evaluation Loss: {average_loss:.4f}')

Evaluation Loss: 0.6580


In [ ]:
category_index = torch.tensor([0], dtype=torch.long)
author_index = torch.tensor([0], dtype=torch.long)
sentiment_index = torch.tensor([0], dtype=torch.long)
title_index = torch.tensor([1], dtype=torch.long)
category_index, author_index, title_index, sentiment_index= category_index.to(device), author_index.to(device), title_index.to(device), sentiment_index.to(device)
prediction = cbf_model(category_index, author_index,sentiment_index,title_index).item()
print(f'Predicted Sentiment: {prediction:.4f}')

Predicted Sentiment: 0.4219


**Сохранение модели**

In [ ]:
model_path = '/content/drive/MyDrive/content_based_filtering_model.pth'

torch.save(cbf_model, model_path)
print(f'Model saved to {model_path}')

Model saved to /content/drive/MyDrive/content_based_filtering_model.pth


In [ ]:
model_path="/content/drive/MyDrive/content_based_filtering_model.pth"
# Загрузить всю модель целиком
cbf_model = torch.load(model_path, map_location=device)
cbf_model.to(device)

<ipython-input-43-0ae4243f3114>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cbf_model = torch.load(model_path, map_location=device)


ContentBasedFilteringModel(
  (category_embedding): Embedding(10669, 64)
  (author_embedding): Embedding(124098, 64)
  (title_embedding): Embedding(206711, 64)
  (sentiment_linear): Linear(in_features=256, out_features=1, bias=True)
)

### **Гибридный подход**

**Получение рекомендаций с использованием  Collaborative Filtering Model**

In [ ]:
def get_collaborative_recommendations(model, title, num_recommendations=100):


    #item_to_index = {title: idx for idx, title in enumerate(item_ids)}
    # Получить индекс входного заголовка
    input_title_index = item_to_index[title]

    # Получение рекомендаций с использованием модели коллаборативной фильтрации
    model.eval()
    with torch.inference_mode():
        # получить похожие заголовки
        similar_titles = model.get_similar_titles(input_title_index, top_k=num_recommendations)


    # возвращаем рекомендуемые заголовки
    return similar_titles

In [ ]:
input_title = "The Universe and Dr. Einstein"
collab_recommendations = get_collaborative_recommendations(model, input_title,num_recommendations=1000)

In [ ]:
collab_recommendations[:10]

['The Universe and Dr. Einstein',
 'Catholic Replies: Answers to over 800 of the most often asked questions about religious and moral issues',
 'Science Fiction Monsters',
 'Alex, the Life of a Child',
 'English for Journalists (Media Skills)',
 'Atlas of Southeast Asia',
 'Food Markets of the World',
 'The hour of the gate',
 'Silly Snacks (Better Homes & Gardens Test Kitchen)',
 "All of This Music Belongs to the Nation: The Wpa's Federal Music Project and American Society"]

**ФИЛЬТРАЦИЯ И СОРТИРОВКА РЕКОМЕНДУЕМОГО СПИСКА КНИГ ПО МОДЕЛИ КОЛЛАБОРАТИВНОЙ ФИЛЬТРАЦИИ С ИСПОЛЬЗОВАНИЕМ МОДЕЛИ КОНТЕНТНОЙ ФИЛЬТРАЦИИ**

In [ ]:
def get_content_based_recommendations(content_based_model, collaborative_recommendations):

    title_details = title_sentiment_aggregated.set_index('Title')[['categories', 'authors', 'sentiment_score']].to_dict(orient='index')

    # Извлечение подробной информации для совместных рекомендаций
    details = [title_details[title] for title in collaborative_recommendations]

    # конвертируем details в тензоры
    category_indices = torch.tensor([category_to_index[detail['categories']] for detail in details], dtype=torch.long)
    author_indices = torch.tensor([author_to_index[detail['authors']] for detail in details], dtype=torch.long)
    title_indices = torch.tensor([title_to_index[title] for title in collaborative_recommendations], dtype=torch.long)
    sentiment_scores = torch.tensor([detail['sentiment_score'] for detail in details], dtype=torch.float32)
    category_indices, author_indices, title_indices, sentiment_scores= category_indices.to(device), author_indices.to(device), title_indices.to(device), sentiment_scores.to(device)

    content_based_model.eval()
    with torch.inference_mode():
        predictions = content_based_model(category_indices, author_indices, title_indices, sentiment_scores)

    # Сортировка заголовков на основе прогнозов
    sorted_titles = [title for _, title in sorted(zip(predictions, collaborative_recommendations), reverse=True)]

    # возвращаем сортированные заголовки
    return sorted_titles

In [ ]:
r=get_content_based_recommendations(cbf_model,collab_recommendations)

In [ ]:
r[:10]

['A Field Guide to Eastern Trees (Peterson Field Guides)',
 "V-1 Flying Bomb 1942-52: Hitler's infamous 'doodlebug' (New Vanguard)",
 "Sorcerers' Apprentices: 100 Years of Law Clerks at the United States Supreme Court",
 'Sixteen Runaway Pumpkins',
 'Brave Jimmy Stone',
 'Chance: A tale in two parts',
 'Fires & Furies: The L. A. Riots- What Really Happened',
 'Dreamrider',
 'Yankee Doodle Boychik',
 'The World Below: A Novel']

In [ ]:
def partial_name_matching(partial_name, all_books):
    matching_titles = [title for title in all_books if partial_name.lower() in title.lower()]

    # используем set для уникальности
    unique_matching_titles = set(matching_titles)

    # форматируем выход в строку
    output_string = "\n".join(unique_matching_titles)

    return output_string

In [ ]:
print( partial_name_matching('introduction to analy',merged['Title'] ))

An Introduction to Analysis and Integration Theory (The Intext Series in Advanced Mathematics)
Introduction to Analysis of Variance: Design, Analyis & Interpretation
An Introduction To Analysis
Yet Another Introduction to Analysis
FORM IN TONAL MUSIC an Introduction to Analysis
Labor and the Economy: An Introduction to Analysis, Issues, and Institutions
Introduction to Analysis


In [ ]:
import ipywidgets as widgets
from IPython.display import display

start_index = 0
items_per_page = 10
last_recommended_name = None  # Чтобы отслеживать изменения в названии рекомендуемой книги


# Создание двух виджетов для ввода текста
partial_name_input = widgets.Text(placeholder='Enter partial book name')
recommended_name_input = widgets.Text(placeholder='Use recommended book name')

# создание кнопок
prev_button = widgets.Button(description='Previous')
next_button = widgets.Button(description='Next')

# Создание кнопки для отображения рекомендаций
show_recommendations_button = widgets.Button(description='Show Recommendations')


# Создание виджета для отображения рекомендаций
output_recommendations = widgets.Output()
output_names = widgets.Output()

# получение рекомендаций
def partial_name_matching(partial_name, all_titles, limit=None):
    # поиск уникальных заголовков, содержащих неполное имя
    matching_titles = set(title for title in all_titles if partial_name.lower() in title.lower())

    matching_titles = list(matching_titles)

    if limit:
        matching_titles = matching_titles[:limit]

    return matching_titles

output_recommendations = widgets.Output()

def on_partial_name_change(change):
    partial_name = change.new
    if partial_name:
        matching_titles = partial_name_matching(partial_name, merged_df['Title'], limit=5)

        with output_names:
            output_names.clear_output(wait=True)
            print("\n".join(matching_titles))

partial_name_input.observe(on_partial_name_change, names='value')

def on_recommended_name_change(change):
    recommended_name = change.new

def on_show_recommendations_button_click(_):
    global last_recommended_name,start_index
    recommended_name = recommended_name_input.value
    if recommended_name :
        if recommended_name != last_recommended_name:
            start_index=0

        recommendations_based_on_recommended_name =get_collaborative_recommendations(model,recommended_name,num_recommendations=1000)

        recommendations_based_on_recommended_name=get_content_based_recommendations(cbf_model,recommendations_based_on_recommended_name)

        last_recommended_name = recommended_name

        recommendations_without_recommended_book = [title for title in recommendations_based_on_recommended_name if title != recommended_name][start_index:start_index+items_per_page]


        with output_recommendations:
            output_recommendations.clear_output(wait=True)
            print("\n".join(recommendations_without_recommended_book))


def on_prev_button_click(_):
    global start_index
    start_index = max(0, start_index - items_per_page)
    on_show_recommendations_button_click(None)

def on_next_button_click(_):
    global start_index
    start_index += items_per_page
    on_show_recommendations_button_click(None)

show_recommendations_button.on_click(on_show_recommendations_button_click)
prev_button.on_click(on_prev_button_click)
next_button.on_click(on_next_button_click)

recommended_name_input.observe(on_recommended_name_change, names='value')

display(partial_name_input)
display(output_names)
display(recommended_name_input)
display(output_recommendations)

display(prev_button)
display(next_button)
display(show_recommendations_button)

Text(value='', placeholder='Enter partial book name')

Output()

Text(value='', placeholder='Use recommended book name')

Output()

Button(description='Previous', style=ButtonStyle())

Button(description='Next', style=ButtonStyle())

Button(description='Show Recommendations', style=ButtonStyle())

In [ ]:
len(merged['categories'].unique())

10669

In [ ]:
import ipywidgets as widgets
from IPython.display import display

partial_category_input = widgets.Text(placeholder='Enter partial category name')

output_filtered_recommendations = widgets.Output()

def on_partial_category_change(change):
    partial_category = change.new

    if partial_category:
        filtered_recommendations = [title for title in merged_df['categories'].unique() if partial_category.lower() in title.lower()][:10]

        with output_filtered_recommendations:
            output_filtered_recommendations.clear_output(wait=True)
            print("\n".join(filtered_recommendations))

partial_category_input.observe(on_partial_category_change, names='value')

display(partial_category_input)
display(output_filtered_recommendations)

Text(value='', placeholder='Enter partial category name')

Output()